In [5]:
# Assignment 5 - Hidden Markov Models

# Import functions from lecture2functions.py and reversecomplement.py
from lecture2functions import loadFASTA
from reversecomplement import revcomp

# Function to find all ORFs longer than the threshold
# Parameter 1) the sequence string: DNA sequence as string
# Parameter 2) & the threshold value: the min ORF length (in codons)
# Output: a list of ORFS, where first # is the first nucleotide position of the start codon, and the second is the first base of the stop codon 
def find_ORFs(seq, threshold):
    start_codon = "ATG"
    stop_codons = {"TAG", "TAA", "TGA"}
    orfs = []
    
    # Iterate through the seq to find start codons
    for i in range(len(seq) - 2):
        codon = seq[i:i+3]
        
        # Check if it's a start codon
        if codon == start_codon:
            for j in range(i + 3, len(seq) - 2, 3):
                stop_codon = seq[j:j+3]
                
                # Check if it's a stop codon
                if stop_codon in stop_codons:
                    orf_length = (j - i) // 3 + 1
                    
                    # Check if ORF length is greater than the threshold
                    if orf_length >= threshold:
                        orfs.append([i + 1, j + 1])  # +1 to make positions 1-based
                    break

    return orfs

# Function to find the longest ORF for each stop codon
# Parameter 1) the sequence string 
# Parameter 2) & the threshold value 
# Output: a list of longest ORFS
def find_longest_ORFs(seq, threshold):
    orfs = find_ORFs(seq, threshold)
    longest_orfs = {}
    
    for orf in orfs:
        start, stop = orf
        codon = seq[stop-3:stop]
        
        # Check if this stop codon has a longer ORF
        if codon not in longest_orfs or (stop - start) > (longest_orfs[codon][1] - longest_orfs[codon][0]):
            longest_orfs[codon] = orf
            
    return list(longest_orfs.values())

# Apply to both forward and reverse strands
# Parameter 1) fasta_file: the path to the FASTA file containing the sequence
# Parameter 2) threshold: the minimum ORF length (in codons) to be considered
# Output: a tuple containing two lists 
def find_ORFs_in_both_strands(fasta_file, threshold):
    sequence = loadFASTA(fasta_file)  # Load sequence from FASTA file
    forward_orfs = find_longest_ORFs(sequence, threshold)
    reverse_sequence = revcomp(sequence)  # Reverse complement for the reverse strand
    reverse_orfs = find_longest_ORFs(reverse_sequence, threshold)
    
    return forward_orfs, reverse_orfs

# Conducting Analysis on 'Coronavirus.fasta"
threshold_value = 100
fasta_file = 'coronavirus.fasta'
forward_orfs, reverse_orfs = find_ORFs_in_both_strands(fasta_file, threshold_value)

# Print ORFs in forward and reverse strands for 'Coronavirus'
print("Forward strand ORFs:", forward_orfs)
print("Reverse strand ORFs:", reverse_orfs)


Forward strand ORFs: [[266, 13481], [13768, 21553], [21536, 25382], [26523, 27189], [27394, 27757], [27894, 28257], [28274, 29531]]
Reverse strand ORFs: [[23415, 23715]]
